# **Feature Engineering Exercise**

- *Darlene Adams*

Your task is to engineer some new features to try to improve a model's ability to predict the total number of bike share rentals during a given hour of the day.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report


pd.set_option('display.max_columns',200)
pd.set_option("display.max_info_rows", 800)
pd.set_option('display.max_info_columns',800)

from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
import warnings
warnings.filterwarnings("ignore")

Import the data the drop the 'casual' and 'registered' columns. These are redundant with your target, 'count'.

## Load the Dataset

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_csv("/content/drive/MyDrive/CodingDojo/05-IntermediateML/Week17/Data/bikeshare_train - bikeshare_train.csv")
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [6]:
df = df.drop(columns=['casual','registered'])
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


Transform the 'datetime' column into a datetime type and use it to create 3 new columns in the data frame containing the:
- Name of the Month
- Name of the Day of the Week
- Hour of the Day

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['datetime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10886 entries, 0 to 10885
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
10886 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 85.2 KB


Make sure all 3 new columns are 'object' datatype so they can be one-hot encoded later.

In [8]:
df['month'] = df['datetime'].dt.month_name()
df['day'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour.astype('object')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,4


Drop the 'datetime' and 'season' columns. These are now redundant.

In [9]:
df = df.drop(columns=['datetime','season'])
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,day,hour
0,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,0
1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,1
2,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,2
3,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,3
4,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,4


The temperatures in the 'temp' and 'atemp' columns are in Celsius. Use `.apply()` and a Lambda function to convert them to Fahrenheit.

In [10]:
df['temp'] = df['temp'].apply(lambda x: (x*9/5) + 32)
df['atemp'] = df['atemp'].apply(lambda x: (x*9/5) + 32)
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,day,hour
0,0,0,1,49.712,57.911,81,0.0,16,January,Saturday,0
1,0,0,1,48.236,56.543,80,0.0,40,January,Saturday,1
2,0,0,1,48.236,56.543,80,0.0,32,January,Saturday,2
3,0,0,1,49.712,57.911,75,0.0,13,January,Saturday,3
4,0,0,1,49.712,57.911,75,0.0,1,January,Saturday,4


Create a new column, 'temp_variance,' which shows how much warmer or colder the current temperature ('temp') is than the average temperate for that day of the year ('atemp'). If the current temperature is warmer than average ('atemp'), the value in 'temp_variance' should be positive.

Drop the 'atemp' column.

In [11]:
df['temp_variance'] = df['temp'] - df['atemp']
df = df.drop(columns='atemp')
df

,holiday,workingday,weather,temp,humidity,windspeed,count,month,day,hour,temp_variance
0,0,0,1,49.712,81,0.0000,16,January,Saturday,0,-8.199
1,0,0,1,48.236,80,0.0000,40,January,Saturday,1,-8.307
2,0,0,1,48.236,80,0.0000,32,January,Saturday,2,-8.307
3,0,0,1,49.712,75,0.0000,13,January,Saturday,3,-8.199
4,0,0,1,49.712,75,0.0000,1,January,Saturday,4,-8.199
...,...,...,...,...,...,...,...,...,...,...,...
10881,0,1,1,60.044,50,26.0027,336,December,Wednesday,19,-7.407
10882,0,1,1,58.568,57,15.0013,241,December,Wednesday,20,-4.797
10883,0,1,1,57.092,61,15.0013,168,December,Wednesday,21,-3.546
10884,0,1,1,57.092,61,6.0032,129,December,Wednesday,22,-6.273


WIll provide optional section as an updated link for feedback at a later date